# Comparison of gene ID mapping between g:convert (g:profiler) and mygene

In [1]:
#Imports
import os, requests
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 300)
pd.set_option("display.max_columns", 100)

import mygene

### Import ENSEMBL IDs

In [5]:
pd.read_csv?

Signature:
pd.read_csv(
    filepath_or_buffer,
    sep=',',
    delimiter=None,
    header='infer',
    names=None,
    index_col=None,
    usecols=None,
    squeeze=False,
    prefix=None,
    mangle_dupe_cols=True,
    dtype=None,
    engine=None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    skiprows=None,
    skipfooter=0,
    nrows=None,
    na_values=None,
    keep_default_na=True,
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    parse_dates=False,
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    iterator=False,
    chunksize=None,
    compression='infer',
    thousands=None,
    decimal=b'.',
    lineterminator=None,
    quotechar='"',
    quoting=0,
    doublequote=True,
    escapechar=None,
    comment=None,
    encoding=None,
    dialect=None,
    tupleize_cols=None,
    error_bad_lines=True,
    warn_bad_lines=True,
    delim_whitespace=False,
    

In [20]:
## Load the gene list
ENSEMBL_IDs = pd.read_csv('ENSEMBL_IDs.csv', squeeze= True)
print(len(ENSEMBL_IDs))
ENSEMBL_IDs.head()

58434


0    ENSG00000000003.14
1     ENSG00000000005.6
2    ENSG00000000419.12
3    ENSG00000000457.14
4    ENSG00000000460.17
Name: Gene, dtype: object

* Trim version in gene IDs (after the dot)

In [14]:
ENSEMBL_trimmed = [ID.split('.')[0] for ID in ENSEMBL_IDs]
ENSEMBL_trimmed[:5]

['ENSG00000000003',
 'ENSG00000000005',
 'ENSG00000000419',
 'ENSG00000000457',
 'ENSG00000000460']

### Map gene IDs to symbols 

* g:convert (g:profiler)

In [33]:
%%time
r = requests.post(
    url='https://biit.cs.ut.ee/gprofiler/api/convert/convert/',
    json={
        'organism':'hsapiens',
        'target':'ENSG',
        'query':ENSEMBL_trimmed,
        }
    )
gconvert = pd.DataFrame(r.json()['result'])

CPU times: user 358 ms, sys: 36.8 ms, total: 394 ms
Wall time: 6.32 s


In [34]:
print(gconvert.shape)
gconvert.head()

(58434, 8)


,converted,description,incoming,n_converted,n_incoming,name,namespaces,query
0,ENSG00000000003,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858],ENSG00000000003,1,1,TSPAN6,"ARRAYEXPRESS,ENSG",query_1
1,ENSG00000000005,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757],ENSG00000000005,1,2,TNMD,"ARRAYEXPRESS,ENSG",query_1
2,ENSG00000000419,dolichyl-phosphate mannosyltransferase subunit...,ENSG00000000419,1,3,DPM1,"ARRAYEXPRESS,ENSG",query_1
3,ENSG00000000457,SCY1 like pseudokinase 3 [Source:HGNC Symbol;A...,ENSG00000000457,1,4,SCYL3,"ARRAYEXPRESS,ENSG",query_1
4,ENSG00000000460,chromosome 1 open reading frame 112 [Source:HG...,ENSG00000000460,1,5,C1orf112,"ARRAYEXPRESS,ENSG",query_1


* mygene

In [17]:
#translate Ensembl ID's to gene symbol
if os.path.exists('ginfo.csv'):
    ginfo = pd.read_csv('ginfo.csv', index_col='Unnamed: 0')
    ens = ENSEMBL_trimmed #feed ensembl IDs without ".xx"
else:
    mg = mygene.MyGeneInfo()
    ens = ENSEMBL_trimmed #feed ensembl IDs without ".xx"
    ginfo = mg.querymany(ens, scopes="ensembl.gene", fields="symbol", species="human", returnall=False, as_dataframe=True, df_index=False)
    ginfo.to_csv('ginfo.csv')

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

In [30]:
print(len(ginfo))
ginfo.head()

58437


,_id,_score,notfound,query,symbol
0,7105,19.005505,NaN,ENSG00000000003,TSPAN6
1,64102,19.618760,NaN,ENSG00000000005,TNMD
2,8813,19.647251,NaN,ENSG00000000419,DPM1
3,57147,20.116043,NaN,ENSG00000000457,SCYL3
4,55732,20.142542,NaN,ENSG00000000460,C1orf112


In [40]:
ginfo[ginfo.duplicated('query', keep =False)]

,_id,_score,notfound,query,symbol
28609,101927745,21.019394,NaN,ENSG00000229425,LOC101927745
28610,105369302,20.876440,NaN,ENSG00000229425,LOC105369302
57809,112268391,6.751291,NaN,ENSG00000285607,LOC112268391
57810,112268392,6.499049,NaN,ENSG00000285607,LOC112268392
57811,112268393,6.499049,NaN,ENSG00000285607,LOC112268393


### Compare